In [4]:
# 連線Cloud sql
import os
from google.cloud.sql.connector import Connector, IPTypes
import pg8000
import sqlalchemy


def connect_with_connector() -> sqlalchemy.engine.base.Engine:
    """
    Initializes a connection pool for a Cloud SQL instance of Postgres.

    Uses the Cloud SQL Python Connector package.
    """
    # Note: Saving credentials in environment variables is convenient, but not
    # secure - consider a more secure solution such as
    # Cloud Secret Manager (https://cloud.google.com/secret-manager) to help
    # keep secrets safe.

    instance_connection_name = "law-chatbot-******:us-central1:law-db"

    db_user = "USER"
    db_pass = "PASSWORD"
    db_name = "test"

    ip_type = IPTypes.PRIVATE if os.environ.get("PRIVATE_IP") else IPTypes.PUBLIC

    # initialize Cloud SQL Python Connector object
    connector = Connector()

    def getconn() -> pg8000.dbapi.Connection:
        conn: pg8000.dbapi.Connection = connector.connect(
            instance_connection_name,
            "pg8000",
            user=db_user,
            password=db_pass,
            db=db_name,
            ip_type=ip_type,
        )
        return conn

    # The Cloud SQL Python Connector can be used with SQLAlchemy
    # using the 'creator' argument to 'create_engine'
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=getconn,
        # ...
    )
    return pool

pool = connect_with_connector()


In [24]:
#寫資料進sql
# connect to connection pool
with pool.connect() as db_conn:
  # create ratings table in our sandwiches database
  db_conn.execute(
    sqlalchemy.text(
      "CREATE TABLE IF NOT EXISTS ratings "
      "( id SERIAL NOT NULL, item VARCHAR(255) NOT NULL, "
      "price FLOAT NOT NULL, category VARCHAR(255) NOT NULL,"
      "rating FLOAT NOT NULL, "
      "PRIMARY KEY (id));"
    )
  )

  # commit transaction (SQLAlchemy v2.X.X is commit as you go)
  db_conn.commit()

  # insert data into our ratings table
  insert_stmt = sqlalchemy.text(
      "INSERT INTO ratings (item, price, category, rating) VALUES (:item, :price, :category, :rating)",
  )

  # insert entries into table
  db_conn.execute(insert_stmt, parameters={"item": "hotdog", "price": 25.0, "category": "junk food" ,"rating": 7.5})
  db_conn.execute(insert_stmt, parameters={"item": "apple", "price": 10.0,"category": "fruit" , "rating": 9.1})
  db_conn.execute(insert_stmt, parameters={"item": "ice cream", "price": 15.0,"category": "junk food" , "rating": 8.3})

  # commit transactions
  db_conn.commit()

  # query and fetch ratings table
  results = db_conn.execute(sqlalchemy.text("SELECT * FROM ratings")).fetchall()

  # show results
  for row in results:
    print(row)

(4, 'hotdog', 25.0, 'junk food', 7.5)
(5, 'apple', 10.0, 'fruit', 9.1)
(6, 'ice cream', 15.0, 'junk food', 8.3)


In [23]:
with pool.connect() as db_conn:
    results = db_conn.execute(sqlalchemy.text("DELETE FROM ratings"))
    db_conn.commit()

In [25]:
results

[(4, 'hotdog', 25.0, 'junk food', 7.5),
 (5, 'apple', 10.0, 'fruit', 9.1),
 (6, 'ice cream', 15.0, 'junk food', 8.3)]

In [26]:
txt = ""
for i in results:
    txt+= ",".join(list(map(lambda x: str(x), i)))+"\n"
print(txt)

4,hotdog,25.0,junk food,7.5
5,apple,10.0,fruit,9.1
6,ice cream,15.0,junk food,8.3



In [32]:
#連線gemini
import vertexai
from vertexai.generative_models import GenerativeModel

# TODO(developer): Update and un-comment below line
# project_id = "PROJECT_ID"

vertexai.init(project="law-chatbot-431613", location="us-central1")
model = GenerativeModel("gemini-1.5-flash-001")

req = "我想要看每個食物的類別為何"
response = model.generate_content(
   ["資料\n",txt,'\n', '使用者需求\n',req,"\n\n","請閱讀資料後，根據使用者需求，以表格方式過濾條件回傳符合條件的結果，並說明為何回傳這些結果"]
)

print(response.text)

I0000 00:00:1722937047.008540 6498714 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


## 符合使用者需求的表格結果：

| 食物 | 類別 |
|---|---|
| hotdog | junk food |
| apple | fruit |
| ice cream | junk food |

## 說明：

使用者要求查看每個食物的類別。 資料中包含食物名稱、價格、類別和評分等信息，但使用者只關注食物和類別。 因此，我們只提取了食物名稱和類別列，並將其整理成表格形式，方便使用者查看。 

